In [ ]:
%%capture
!pip install --upgrade kaggle_environments

In [ ]:
from kaggle_environments import make
env = make("kore_fleets", debug=True)
env.run(["../input/kore-intro-iii-expanding-the-empire-unofficial/expander3.py"])
env.render(mode="ipython", width=700, height=700)

pixyz  
last update 2022 05 21  
ゆっくりしていってね！  

こっちも観てね！

Kore Intro I: unofficial  
https://www.kaggle.com/code/pixyz0130/kore-intro-i-the-basics

Kore Intro II: unofficial  
https://www.kaggle.com/code/pixyz0130/kore-intro-ii-mining-kore

**霊夢:今回はコンペ理解のために、主催者のBovard氏が書いてくれたコードを日本語訳してみたよ。今回はIntro Ⅲをやっていくよ。**

**魔理沙:他に気になったことがあったら突っ込んでいくぞ。**

**Reimu: This time, I translated the code written by the organizer into Japanese to understand the competition.**

**Marisa: If you have any other concerns, I'll dig in.**

<img src="https://4.bp.blogspot.com/-uoVuBWIbdiA/WvQHqpx_YCI/AAAAAAABL8g/NiFZ6K71VBc_0_dcKb3_4nhnvFJ_JMNuACLcBGAs/s450/network_dennou_sekai_figure.png" width = 300>

# Contents

* v0 [**Official guide**](#Official-guide)

* v1 [**Optimal number of shipyard**](#Optimal-number-of-shipyard)

# Official guide

**霊夢:まずは公式ガイドの日本語訳からやってくよ**  
https://www.kaggle.com/code/bovard/kore-intro-iii-expanding-the-empire

# Welcome back Commander!


By this time, you know how fleets move and how to mine kore. Next we'll learn how to expand your empire and make new shipyards!

To create a shipyard, you need a fleet with at least 50 (`config.convert_cost`) that has a flight plan that includes the `C` (convert) command.

これまでに、あなたは艦隊がどのように動くか、そしてどのように資源を回収するかを学習しました。 次に、国を拡大して新しい造船所を作る方法を学びます！

造船所を作成するには、 `C`コマンドを含む飛行計画を持つ少なくとも50（`config.convert_cost`）の船が必要です。

> When converting into a shipyard, any kore on the cell underneath the new shipyard will be destroyed. Any kore carried by the fleet converting itself will be deposited instantly and the fleet will lose 50 ships. Any remaining ships are added to the shipyard’s ship count.

>造船所に変化させると、新しい造船所のマスにあった資源はすべて破壊されます。 造船所に変化する前の艦隊によって運ばれていたKoreは即座に預けられ、艦隊は50隻の船を失います。 残りの船は造船所の船数に追加されます。

Let's start with our pilot bot from [part 1](https://www.kaggle.com/bovard/kore-fleets-intro-part-1-the-basics?scriptVersionId=90399125&cellId=15)

In [ ]:
%%writefile pilot.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        if shipyard.ship_count >= 50:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(50, flight_plan)
            shipyard.next_action = action
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
            shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/pilot.py"])
env.render(mode="ipython", width=1000, height=800)

This bot will allow us to gather enough resources to found a new shipyard without running out of core! Let's gate the creation of a new shipyard to when we have enough kore to build another fleet: 500.

このボットを使用すると、資源を使い果たすことなく、新しい造船所を建設するのに十分な資源を集めることができます。 新しい造船所の作成を、別の艦隊を進水させるのに十分な資源があるときに命令してみましょう。

## Expander

In [ ]:
%%writefile expander.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint

# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander.py"])
env.render(mode="ipython", width=1000, height=800)

**魔理沙:造船所を作るには、50隻もの船を消費しなきゃいけないんだぜ。**

**霊夢:Koreにすると500個だね、結構多いなあ。**

**魔理沙:造船所に変化する艦隊が持っていた資源は回収されて、艦隊の船の数が50を超えていた場合、超えていた分は新しくできた造船所に保管されるはずだぜ。**

**Marisa: You have to consume as many as 50 ships to build a shipyard.**

**Reimu: Kore has 500, which is quite a lot.**

**Marisa: The resources of the changing fleet at the shipyard will be recovered, and if the number of ships in the fleet exceeds 50, the excess should be stored in the new shipyard.**

# Optimal number of shipyard

**霊夢:造船所を作るには船を50隻、Koreになおすと500Kore消費するんだよね。造船所はどのくらい作った方がいいのかな？**

**魔理沙:じゃあ実際に造船所を作る数を変化させて結果を観察してみよう**

**Reimu: To build a shipyard, you need 50 ships, and if you convert it to Kore, you will consume 500 Kore. How long should a shipyard be built?**

**Marisa: Let's change the number of shipyards actually built and observe the results**

**造船所の最大数をそれぞれ1個、2個、4個、6個、8個にして実験をしてみます。**

**Experiment with the maximum number of shipyards of 1, 2, 4, 6, and 8 respectively.**

In [ ]:
env = make("kore_fleets",configuration={"randomSeed":42}, debug=True)

## 1 shipyard

In [ ]:
%%writefile expander2.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import random
# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    random.seed(turn)
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore

    for shipyard in me.shipyards:
        action = None
        if shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander2.py"])
env.render(mode="ipython", width=1000, height=800)

## 2 shipyards

In [ ]:
%%writefile expander3.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import random
# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):

    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    random.seed(turn)
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore
    shipyards_size = len(me.shipyards)
    max_shipyards = 2
    
    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost+10 and shipyards_size < max_shipyards:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost+10, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander3.py"])
env.render(mode="ipython", width=1000, height=800)

## 4 shipyards

In [ ]:
%%writefile expander4.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import random
# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):

    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    random.seed(turn)
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore
    shipyards_size = len(me.shipyards)
    max_shipyards = 4
    
    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost+10 and shipyards_size < max_shipyards:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost+10, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander4.py"])
env.render(mode="ipython", width=1000, height=800)

## 6 shipyards

In [ ]:
%%writefile expander5.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import random
# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):

    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    random.seed(turn)
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore
    shipyards_size = len(me.shipyards)
    max_shipyards = 6
    
    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost+10 and shipyards_size < max_shipyards:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost+10, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander5.py"])
env.render(mode="ipython", width=1000, height=800)

## 8 shipyards

In [ ]:
%%writefile expander6.py
   
from kaggle_environments.envs.kore_fleets.helpers import *
from random import randint
import random
# a flight plan
def build_flight_plan(dir_idx, size):
    flight_plan = ""
    for i in range(4):
        flight_plan += Direction.from_index((dir_idx + i) % 4).to_char()
        if not i == 3:
            flight_plan += str(size)
    return flight_plan

def agent(obs, config):
    board = Board(obs, config)
    me=board.current_player

    me = board.current_player
    turn = board.step
    random.seed(turn)
    spawn_cost = board.configuration.spawn_cost
    convert_cost = board.configuration.convert_cost
    kore_left = me.kore
    shipyards_size = len(me.shipyards)
    max_shipyards = 8
    
    for shipyard in me.shipyards:
        action = None
        if kore_left >= 500 and shipyard.ship_count >= convert_cost+10 and shipyards_size < max_shipyards:
            flight_plan = build_flight_plan(randint(0, 3), randint(10, 15))
            flight_plan = flight_plan[:6] + "C"
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost+10, flight_plan)
        elif shipyard.ship_count >= convert_cost:
            flight_plan = build_flight_plan(randint(0, 3), randint(2, 9))
            action = ShipyardAction.launch_fleet_with_flight_plan(convert_cost, flight_plan)
        elif kore_left >= spawn_cost:
            action = ShipyardAction.spawn_ships(1)
        shipyard.next_action = action

    return me.next_actions

In [ ]:
env.run(["/kaggle/working/expander6.py"])
env.render(mode="ipython", width=1000, height=800)

**randomSeedを41に設定して、違うマップでも実験をしてみます。**

In [ ]:
env = make("kore_fleets",configuration={"randomSeed":41}, debug=True)

## 1 shipyard

In [ ]:
env.run(["/kaggle/working/expander2.py"])
env.render(mode="ipython", width=1000, height=800)

## 2 shipyard

In [ ]:
env.run(["/kaggle/working/expander3.py"])
env.render(mode="ipython", width=1000, height=800)

## 4 shipyard

In [ ]:
env.run(["/kaggle/working/expander4.py"])
env.render(mode="ipython", width=1000, height=800)

## 6 shipyard

In [ ]:
env.run(["/kaggle/working/expander5.py"])
env.render(mode="ipython", width=1000, height=800)

## 8 shipyard

In [ ]:
env.run(["/kaggle/working/expander6.py"])
env.render(mode="ipython", width=1000, height=800)

**霊夢:結果を表にまとめてみたよ**

**Reimu: I've summarized the results in a table**

randomSeed = 42  

| &nbsp;&nbsp;Shipyard size &nbsp;&nbsp;| &nbsp;&nbsp;&nbsp;Kore&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;Cargo&nbsp;&nbsp; | &nbsp;&nbsp;Ships&nbsp;&nbsp; |  
| :---: | ---- | ---- | ---- |
| 1 | **23994** | 4103 | 323 |  
| 2 | **21775** | 5138 | 615 |  
| 4 | **20361** | 4928 | 704 |  
| 6 | **17108** | 2998 | 802 |  
| 8 | **13530** | 3734 | 691 |  

randomSeed = 41 

| &nbsp;&nbsp;Shipyard size &nbsp;&nbsp;| &nbsp;&nbsp;&nbsp;Kore&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;Cargo&nbsp;&nbsp; | &nbsp;&nbsp;Ships&nbsp;&nbsp; |  
| :---: | ---- | ---- | ---- |
| 1 | **28893** | 3985 | 309 |  
| 2 | **29135** | 6945 | 595 |  
| 4 | **28200** | 4333 | 733 |  
| 6 | **20543** | 6532 | 778 |  
| 8 | **14080** | 3411 | 742 |  

**魔理沙:造船した数も踏まえると、造船所を2～4個作ると資源を多く集めれそうだね。**

**霊夢:味方しかいない場合はこういう結果だったけど、敵がいるときも同じ結果になるのかな？**

**魔理沙:どうなんだろう？、次回はIntroⅣ、敵と戦闘について学んでいくぞ**

**Marisa: Considering the number of ships built, it seems that you can collect a lot of resources by building 2 to 4 shipyards.**

**Reimu: This was the result when there were only allies, but is it the same result when there are enemies?**

**Marisa: What about? Next time, Intro IV, I'll learn about fighting with enemies**